In [31]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import time
from sklearn.cross_validation import train_test_split

C:\Users\Cheong\Anaconda3\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
customers = pd.read_csv("data/olist_customers_dataset.csv")
orders = pd.read_csv("data/olist_orders_dataset.csv")
items = pd.read_csv("data/olist_order_items_dataset.csv")

In [3]:
data = items.merge(orders, on = "order_id")
data = data.merge(customers, on = "customer_id").drop('customer_id', axis=1)
data.head()

,order_id,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state
0,00010242fe8c5a6d1ba2dd792cb16214,1,4244733e06e7ecb4970a6e2683c13e61,48436dade18ac8b2bce089ec2a041202,2017-09-19 09:45:35,58.90,13.29,delivered,2017-09-13 08:59:02,2017-09-13 09:45:35,2017-09-19 18:34:16,2017-09-20 23:43:48,2017-09-29 00:00:00,871766c5855e863f6eccc05f988b23cb,28013,campos dos goytacazes,RJ
1,00018f77f2f0320c557190d7a144bdd3,1,e5f2d52b802189ee658865ca93d83a8f,dd7ddc04e1b6c2c614352b383efe2d36,2017-05-03 11:05:13,239.90,19.93,delivered,2017-04-26 10:53:06,2017-04-26 11:05:13,2017-05-04 14:35:00,2017-05-12 16:04:24,2017-05-15 00:00:00,eb28e67c4c0b83846050ddfb8a35d051,15775,santa fe do sul,SP
2,000229ec398224ef6ca0657da4fc703e,1,c777355d18b72b67abbeef9df44fd0fd,5b51032eddd242adc84c38acab88f23d,2018-01-18 14:48:30,199.00,17.87,delivered,2018-01-14 14:33:31,2018-01-14 14:48:30,2018-01-16 12:36:48,2018-01-22 13:19:16,2018-02-05 00:00:00,3818d81c6709e39d06b2738a8d3a2474,35661,para de minas,MG
3,00024acbcdf0a6daa1e931b038114c75,1,7634da152a4610f1595efa32f14722fc,9d7a1d34a5052409006425275ba1c2b4,2018-08-15 10:10:18,12.99,12.79,delivered,2018-08-08 10:00:35,2018-08-08 10:10:18,2018-08-10 13:28:00,2018-08-14 13:32:39,2018-08-20 00:00:00,af861d436cfc08b2c2ddefd0ba074622,12952,atibaia,SP
4,00042b26cf59d7ce69dfabb4e55b4fd9,1,ac6c3623068f30de03045865e4e10089,df560393f3a51e74553ab94004ba5c87,2017-02-13 13:57:51,199.90,18.14,delivered,2017-02-04 13:57:51,2017-02-04 14:10:13,2017-02-16 09:46:09,2017-03-01 16:42:31,2017-03-17 00:00:00,64b576fb70d441e8f1b2d7d446e483c5,13226,varzea paulista,SP


In [23]:
## For each customer ID, get the products they purchased and the number of times they purchased that product

df = pd.melt(data, 
             id_vars=['customer_unique_id'],
             value_vars=['product_id']).drop(['variable'], axis=1) \
            .rename(columns={'value': 'product_id'}) \
            .groupby(['customer_unique_id', 'product_id'])\
            .agg({'product_id': 'count'}) \
            .rename(columns={'product_id': 'purchase_count'}) \
            .reset_index()
            
df.head()


,customer_unique_id,product_id,purchase_count
0,0000366f3b9a7992bf8c76cfdf3221e2,372645c7439f9661fbbacfd129aa92ec,1
1,0000b849f77a49e4a4ce2b2a4ca5be3f,5099f7000472b634fea8304448d20825,1
2,0000f46a3911fa3c0805444483337064,64b488de448a5324c4134ea39c28a34b,1
3,0000f6ccb0745a6a4b88665a16c9f078,2345a354a6f2033609bbf62bf5be9ef6,1
4,0004aac84e0df4da2b147fca70cf8255,c72e18b3fe2739b8d24ebf3102450f37,1


In [24]:
## Create dummy for whether the customer bought that product or not

def create_data_dummy(data):
    data_dummy = data.copy()
    data_dummy['purchase_dummy'] = 1
    return data_dummy
data_dummy = create_data_dummy(df)

data_dummy.head()

,customer_unique_id,product_id,purchase_count,purchase_dummy
0,0000366f3b9a7992bf8c76cfdf3221e2,372645c7439f9661fbbacfd129aa92ec,1,1
1,0000b849f77a49e4a4ce2b2a4ca5be3f,5099f7000472b634fea8304448d20825,1,1
2,0000f46a3911fa3c0805444483337064,64b488de448a5324c4134ea39c28a34b,1,1
3,0000f6ccb0745a6a4b88665a16c9f078,2345a354a6f2033609bbf62bf5be9ef6,1,1
4,0004aac84e0df4da2b147fca70cf8255,c72e18b3fe2739b8d24ebf3102450f37,1,1


In [72]:
## Affinity score: Higher score if more customers buy both of the products

#Get list of unique products
productList=list(set(df["product_id"].tolist()))

#Get count of customers
custCount=len(set(df["customer_unique_id"].tolist()))

#Create an empty data frame to store item affinity scores for items.
itemAffinity= pd.DataFrame(columns=('item1', 'item2', 'score'))
rowCount=0

In [77]:
searchItem = "372645c7439f9661fbbacfd129aa92ec"

#For each product in the product list, compare with other products to get the affinity score.
for ind1 in range(len(productList)):
    
    if productList[ind1] != searchItem:
        continue
            
    #Get list of users who bought this item 1.
    item1Users = df[df.product_id==productList[ind1]]["customer_unique_id"].tolist()
    #print("Item 1 ", item1Users)
    
    #Get item 2 - items that are not item 1 or those that are not analyzed already.
    for ind2 in range(ind1, len(productList)):
        
        if ( ind1 == ind2):
            continue
                   
        #Get list of users who bought item 2
        item2Users=df[df.product_id==productList[ind2]]["customer_unique_id"].tolist()
        #print("Item 2",item2Users)
        
        #Find score. Find the common list of users and divide it by the total users.
        commonUsers= len(set(item1Users).intersection(set(item2Users)))
        score=commonUsers 

        #Add a score for item 1, item 2
        itemAffinity.loc[rowCount] = [productList[ind1],productList[ind2],score]
        rowCount +=1

In [78]:
## Sort items to recommend in descending order

recoList=itemAffinity[itemAffinity.item1==searchItem]\
        [["item2","score"]]\
        .sort_values("score", ascending=[0])
        
## Remove zero scores
recoList = recoList[recoList["score"] != 0]
        
print("Recommendations for", searchItem, "\n", recoList)


Recommendations for 372645c7439f9661fbbacfd129aa92ec 
                                   item2    score
31044  42155695adbe665066ad812855fe523a  1.00000
35188  525947dbe3304ac32bf51602f9557c12  1.00000
27314  bd1aace4fad5609f005fd721b45dcec4  1.00000
29220  cc1b0e67ffb98a08c886e8c3c27a915f  1.00000
26635  3ea30fb7b4c6d17f44c1594a713c224c  1.00000
30182  2b10e945dae5434075c8bb2be0d17325  1.00000
10264  cc1b0e67ffb98a08c886e8c3c27a915f  0.00001
5094   3ea30fb7b4c6d17f44c1594a713c224c  0.00001
22200  525947dbe3304ac32bf51602f9557c12  0.00001
6452   bd1aace4fad5609f005fd721b45dcec4  0.00001
13912  42155695adbe665066ad812855fe523a  0.00001
12188  2b10e945dae5434075c8bb2be0d17325  0.00001


In [71]:
################# Deprecated ###############################

#Get list of unique customers
custList=list(set(df["customer_unique_id"].tolist()))

#For each customer in the customer list, get the products that he purchased, and add to the affinity score.
for ind in range(len(custList)):
            
    #Get list of products that the customer bought.
    products = df[df.customer_unique_id==custList[ind]]["product_id"].tolist()
    
    #Add to affinity score
    for ind1 in range(len(products)):
        for ind2 in range(len(products)):
            
            if ( ind1 == ind2):
                continue
                
            if itemAffinity[(itemAffinity.item1==products[ind1]) & (itemAffinity.item2 == products[ind2])].shape[0] == 0:

                #Add a score for item 1, item 2
                itemAffinity.loc[rowCount] = [products[ind1],products[ind2],1]
                rowCount +=1
                #Add a score for item2, item 1. The same score would apply irrespective of the sequence.
                itemAffinity.loc[rowCount] = [products[ind2],products[ind1],1]
                rowCount +=1
                
            else:
                
                #Add to existing row
                itemAffinity[(itemAffinity.item1==products[ind1]) & (itemAffinity.item2 == products[ind2])]['score'] += 1
        
#Check final result
itemAffinity.head()

C:\Users\Cheong\Anaconda3\lib\site-packages\ipykernel_launcher.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


KeyboardInterrupt: 